In [1]:
# import libraries and packages
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

In [1]:
# set root directory
ROOT_DIR = os.path.abspath("../")

# import mask r-cnn library
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# find path of default coco weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# create logs directory to store custom weights
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

/Users/samiagilani/opt/anaconda3/envs/mask_rcnn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/samiagilani/opt/anaconda3/envs/mask_rcnn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/samiagilani/opt/anaconda3/envs/mask_rcnn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/

In [2]:
# training configurations
class RootConfig(Config):

    # set configuration name
    NAME = "root"

    # set number of images to train at a time
    IMAGES_PER_GPU = 1

    # set number of classes
    NUM_CLASSES = 1 + 1  # background + root

    # set number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # set the minimum detection confidence level
    DETECTION_MIN_CONFIDENCE = 0.9

In [3]:
# root dataset
class RootDataset(utils.Dataset):

    def load_root(self, dataset_dir, subset):

        # append "root" class
        self.add_class("root", 1, "root")

        # check which dataset is being used
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # load annotations from file
        annotations = json.load(open(os.path.join(dataset_dir, "annotations.json")))
        annotations = list(annotations.values())
        annotations = [a for a in annotations if a['regions']]

        # retrieve coordinates from polygons
        for a in annotations:
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']] 

            # calculate image size
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            # add image
            self.add_image(
                "root",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons)

    # create masks for each image
    def load_mask(self, image_id):
        image_info = self.image_info[image_id]
        if image_info["source"] != "root":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # return mask
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    # return image path
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        if info["source"] == "root":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [22]:
# train
def train(model):
    # train dataset
    dataset_train = RootDataset()
    dataset_train.load_root("../dataset", "train")
    dataset_train.prepare()

    # val dataset
    dataset_val = RootDataset()
    dataset_val.load_root("../dataset", "val")
    dataset_val.prepare()

    # train network heads -- should be sufficient to train model
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads')

In [7]:
# set configurations
config = RootConfig()
config.display()

In [11]:
# create model
model = modellib.MaskRCNN(mode="training", config=config, model_dir=os.path.join(ROOT_DIR, "logs"))

In [13]:
# load in default coco weights
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
# train model
train(model)